In [ ]:
import cv2
import numpy as np

video_path = "prueba11.mp4" 
cap = cv2.VideoCapture(video_path)

ret, first_frame = cap.read()
if not ret:
    print("Error al cargar el video.")
    exit()

resize_factor = 0.5  

prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
mask = np.zeros_like(first_frame)

maxCorners = 100
qualityLevel = 0.5
minDistance = 100
blockSize = 7

# Lucas-Kanade
winSize = (15, 15)
maxLevel = 2
criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)

# HSV for blue
lower_bound = np.array([112, 132, 129])  
upper_bound = np.array([115, 251, 176])  

hsv = cv2.cvtColor(first_frame, cv2.COLOR_BGR2HSV)
blue_mask = cv2.inRange(hsv, lower_bound, upper_bound)

p0 = cv2.goodFeaturesToTrack(prev_gray, mask=blue_mask, maxCorners=maxCorners,
                              qualityLevel=qualityLevel, minDistance=minDistance, blockSize=blockSize)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
  
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    blue_mask = cv2.inRange(hsv, lower_bound, upper_bound)

    if p0 is not None:
        p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, p0, None,
                                               winSize=winSize, maxLevel=maxLevel, criteria=criteria)
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel().astype(int)
            c, d = old.ravel().astype(int)
            frame = cv2.circle(frame, (a, b), 5, (255, 0, 0), -1)  
            mask = cv2.line(mask, (a, b), (c, d), (255, 0, 0), 2)  

        p0 = good_new.reshape(-1, 1, 2)
    else:
        p0 = cv2.goodFeaturesToTrack(frame_gray, mask=blue_mask, maxCorners=maxCorners,
                                     qualityLevel=qualityLevel, minDistance=minDistance, blockSize=blockSize)

    prev_gray = frame_gray.copy()
    frame_resized = cv2.resize(frame, None, fx=resize_factor, fy=resize_factor)
    mask_resized = cv2.resize(mask, None, fx=resize_factor, fy=resize_factor)
    output = cv2.add(frame_resized, mask_resized)
    cv2.imshow("Optical Flow", output)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()